In [33]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.graph_objects as go

import shutil
import random

from scipy import signal
from collections import OrderedDict

In [34]:
path = r'c:\Users\dhers\OneDrive - UBA\DLC\STORM-sdhers-2024-10-01\results\Original_TS_A2024-09-27T16_14_10_LDLC_Resnet50_STORMOct1shuffle1_snapshot_200.h5'

df = pd.read_hdf(path)
scorer = df.columns.levels[0][0]
bodyparts = df.columns.levels[1].to_list()

df_raw = df.copy()
df

scorer    DLC_Resnet50_STORMOct1shuffle1_snapshot_200                         \
bodyparts                                       obj_1                          
coords                                              x           y likelihood   
0                                          218.831787  133.034561   0.958618   
1                                          218.882553  133.019897   0.957685   
2                                          218.851120  133.027664   0.958130   
3                                          218.869156  133.030380   0.958193   
4                                          218.848404  133.018417   0.958241   
...                                               ...         ...        ...   
8995                                       218.172150  133.573074   0.954947   
8996                                       218.170868  133.584808   0.955158   
8997                                       218.145477  133.580673   0.954891   
8998                                       218.164932  133.580627   0.954687   
8999                                       218.152130  133.567780   0.955133   

scorer                                                                \
bodyparts       obj_2                               nose               
coords              x           y likelihood           x           y   
0          486.046478  136.649765   0.951709   69.788048  318.935455   
1          486.079865  136.719193   0.953084   69.822632  319.009186   
2          486.071228  136.694153   0.952720   69.811249  319.038757   
3          486.024384  136.655350   0.952345   69.819710  319.068237   
4          486.051361  136.615891   0.951124   69.789856  318.985992   
...               ...         ...        ...         ...         ...   
8995       485.258667  136.432205   0.959699  109.891190  172.790359   
8996       485.236633  136.424072   0.960046  110.026230  172.485397   
8997       485.261322  136.402023   0.959627  110.072701  171.904007   
8998       485.217712  136.403458   0.959445  109.840775  171.898193   
8999       485.279205  136.397308   0.959267  109.881828  172.657547   

scorer                           ...                                     \
bodyparts                 L_ear  ...       body      tail_1               
coords    likelihood          x  ... likelihood           x           y   
0           0.015600  72.574989  ...   0.008184   81.755737  330.704773   
1           0.016262  72.598709  ...   0.008088   81.888985  330.861237   
2           0.016570  72.586296  ...   0.007953   81.979935  330.819824   
3           0.016536  72.586296  ...   0.007992   81.986526  330.872650   
4           0.016051  72.559219  ...   0.008023   81.956558  330.902863   
...              ...        ...  ...        ...         ...         ...   
8995        0.923901  86.063919  ...   0.876708  102.585579  339.881104   
8996        0.921804  86.097542  ...   0.881453  102.590004  339.879242   
8997        0.920993  86.333275  ...   0.876992  102.573830  339.847412   
8998        0.926821  86.171333  ...   0.881875  102.595581  339.859131   
8999        0.922490  86.127594  ...   0.874267  102.581429  339.869110   

scorer                                                               \
bodyparts                 tail_2                             tail_3   
coords    likelihood           x           y likelihood           x   
0           0.019286   69.876083  338.011749   0.154845   73.829453   
1           0.018462   70.162285  337.657013   0.162661   73.860489   
2           0.017199   74.946899  334.273376   0.173637   73.761086   
3           0.016747   74.913071  334.084106   0.173278   73.773987   
4           0.016426   71.022667  337.574585   0.182343   73.556778   
...              ...         ...         ...        ...         ...   
8995        0.962702  153.642654  383.364227   0.918964  222.194351   
8996        0.962711  153.637772  383.347107   0.921220  222.139999   
8997        0.964247  153.899200  383.

In [35]:
# Try different filtering parameters
window, sigma, n_sigmas = 21, 1, 2
N = int(2 * n_sigmas * sigma + 1)

# Gaussian kernel
kernel = signal.windows.gaussian(N, sigma)
kernel = kernel / sum(kernel)

pad_width = (len(kernel) - 1) // 2

In [36]:
# Iterate over each body part
for point in bodyparts:

    mean = df[scorer, point, 'likelihood'].mean()
    std_dev = df[scorer, point, 'likelihood'].std()
    limit = max(0.1, mean - 0*std_dev)

    print(f'{point}\t{limit}')
    
    # Create mask where likelihood is below the threshold
    mask = df[scorer, point, 'likelihood'] < limit
    # Set 'x' and 'y' to NaN where likelihood is below the threshold
    df.loc[mask, (scorer, point, ['x', 'y'])] = np.nan

    # Iterate over x and y axes
    for axis in ['x', 'y']:
        # Access the specific axis column for this body part
        # column = df[scorer, point, axis]

        # Check if there are any non-NaN values left to interpolate
        if df[scorer, point, axis].notna().sum() > 1:
            # Interpolate using the pchip method, limit_area='inside' ensures interpolation only within valid data
            df[scorer, point, axis] = df[scorer, point, axis].interpolate(method='pchip', limit_area='inside')
            
            # Forward fill the remaining NaN values
            df[scorer, point, axis] = df[scorer, point, axis].ffill().bfill()
            
            # Apply median filter
            df[scorer, point, axis] = signal.medfilt(df[scorer, point, axis], kernel_size=window)
            
            # Pad the median filtered data to mitigate edge effects
            padded = np.pad(df[scorer, point, axis], pad_width, mode='edge')
            
            # Apply convolution
            smooth = signal.convolve(padded, kernel, mode='valid')
            
            # Trim the padded edges to restore original length
            df[scorer, point, axis] = smooth[:len(df[scorer, point, axis])]
        
            if 'obj' in point:
                df[scorer, point, axis] = df[scorer, point, axis].median()
            


df

L_ear	0.9337078928947449
R_ear	0.9356409311294556
body	0.929391622543335
head	0.904685378074646
neck	0.9255021810531616
nose	0.865659236907959
obj_1	0.9562800526618958
obj_2	0.9383792281150818
tail_1	0.9300345182418823
tail_2	0.9206780791282654
tail_3	0.9285717010498047


scorer    DLC_Resnet50_STORMOct1shuffle1_snapshot_200                         \
bodyparts                                       obj_1                          
coords                                              x           y likelihood   
0                                          218.271367  133.484558   0.958618   
1                                          218.271367  133.484558   0.957685   
2                                          218.271367  133.484558   0.958130   
3                                          218.271367  133.484558   0.958193   
4                                          218.271367  133.484558   0.958241   
...                                               ...         ...        ...   
8995                                       218.271367  133.484558   0.954947   
8996                                       218.271367  133.484558   0.955158   
8997                                       218.271367  133.484558   0.954891   
8998                                       218.271367  133.484558   0.954687   
8999                                       218.271367  133.484558   0.955133   

scorer                                                               \
bodyparts       obj_2                              nose               
coords              x          y likelihood           x           y   
0          485.278602  136.32189   0.951709  211.272293  243.352890   
1          485.278602  136.32189   0.953084  211.272293  243.352890   
2          485.278602  136.32189   0.952720  211.272293  243.352890   
3          485.278602  136.32189   0.952345  211.272293  243.352890   
4          485.278602  136.32189   0.951124  211.272293  243.352890   
...               ...        ...        ...         ...         ...   
8995       485.278602  136.32189   0.959699  109.929024  172.552996   
8996       485.278602  136.32189   0.960046  109.879141  172.473324   
8997       485.278602  136.32189   0.959627  109.847717  172.303205   
8998       485.278602  136.32189   0.959445  109.821003  172.067889   
8999       485.278602  136.32189   0.959267  109.796552  171.929816   

scorer                            ...                                     \
bodyparts                  L_ear  ...       body      tail_1               
coords    likelihood           x  ... likelihood           x           y   
0           0.015600  572.522095  ...   0.008184  447.943787  368.693390   
1           0.016262  572.522095  ...   0.008088  447.943787  368.693390   
2           0.016570  572.522095  ...   0.007953  447.943787  368.693390   
3           0.016536  572.522095  ...   0.007992  447.943787  368.693390   
4           0.016051  572.522095  ...   0.008023  447.943787  368.693390   
...              ...         ...  ...        ...         ...         ...   
8995        0.923901   86.148236  ...   0.876708  102.580265  339.867925   
8996        0.921804   86.125601  ...   0.881453  102.574052  339.865034   
8997        0.920993   86.105835  ...   0.876992  102.568051  339.860661   
8998        0.926821   86.089222  ...   0.881875  102.563586  339.855749   
8999        0.922490   86.073981  ...   0.874267  102.560994  339.850741   

scorer                                                               \
bodyparts                 tail_2                             tail_3   
coords    likelihood           x           y likelihood           x   
0           0.019286  391.892365  386.874084   0.154845  306.493378   
1           0.018462  391.892365  386.874084   0.162661  306.493378   
2           0.017199  391.892365  386.874084   0.173637  306.493378   
3           0.016747  391.892365  386.874084   0.173278  306.493378   
4           0.016426  391.892365  386.874084   0.182343  306.493378   
...              ...         ...         ...        ...         ...   
8995        0.962702  153.605301  383.331665   0.918964  222.192949   
8996        0.962711  153.605301  383.331665   0.921220  222.184775   
8997        0.964247  153.605301  383.

In [37]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=df.index, y=df[scorer]['nose']['x']))
fig.add_trace(go.Scatter(x=df_raw.index, y=df_raw[scorer]['nose']['x']))

In [38]:
# Save the updated DataFrame to an HDF5 file
out_path = r'c:\Users\dhers\OneDrive - UBA\DLC\STORM-sdhers-2024-10-01\results\TS_A2024-09-27T16_14_10_LDLC_Resnet50_STORMOct1shuffle1_snapshot_200.h5'
df.to_hdf(out_path, key='df', mode='w')